<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/mm-imdb(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/Dataset(s)/mm-imdb/fused

/content/Dataset(s)/mm-imdb/fused


In [ ]:
!gdown https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X

Downloading...
From: https://drive.google.com/uc?id=1dxd2pySfCIDJYG7qMtuJre8ph068xc1X
To: /content/Dataset(s)/mm-imdb/fused/test.zip
100% 2.12G/2.12G [00:26<00:00, 79.8MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ

Downloading...
From: https://drive.google.com/uc?id=1sYR9EgHkM0oiGRQVlFQCyHO8kMRJ4ibQ
To: /content/Dataset(s)/mm-imdb/fused/test_label.txt
100% 777k/777k [00:00<00:00, 8.16MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP

Downloading...
From: https://drive.google.com/uc?id=1X4cmMYRjxXFomCJ1adMhPNMYtd4WeDHP
To: /content/Dataset(s)/mm-imdb/fused/train_label.txt
100% 1.55M/1.55M [00:00<00:00, 14.6MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq

Downloading...
From: https://drive.google.com/uc?id=1yAeNEPXD3LNxmtSnC09jEg0vEDc1qiaq
To: /content/Dataset(s)/mm-imdb/fused/train.zip
100% 4.21G/4.21G [00:50<00:00, 83.8MB/s]


In [ ]:
!unzip test.zip

In [ ]:
!unzip train.zip

# New Section

In [ ]:
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        self.data_file = data_file
        self.root_dir = root_dir
        self.transform = transform

        with open(data_file, 'r') as f:
            self.data = f.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        line = self.data[idx].strip().split('|')
        image_path = line[0].strip()
        image = Image.open(os.path.join(self.root_dir, image_path))

        labels = line[1].strip().split(',')
        labels = [label.strip() for label in labels]

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = CustomDataset(data_file='/content/test_label.txt', root_dir='/content/test', transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
with open('/content/test_label.txt', 'r') as f:
    lines = f.readlines()

unique_labels = set()
for line in lines:
    labels = line.strip().split('|')[1].strip().split(',')
    unique_labels.update(labels)

num_classes = len(unique_labels)
print(f"Number of classes: {num_classes}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define your ResNet-101 model
model = models.resnet101(pretrained=True)  # You can use a pre-trained model

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes),
    nn.Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}')

print('Training finished.')

# Now, you can use the trained model for evaluation and prediction
# Don't forget to create a DataLoader for your test/validation dataset and evaluate the model's performance.


# Training Dataset Preparation

In [ ]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "/content/Dataset(s)/mm-imdb/fused/train"
labels_file = '/content/Dataset(s)/mm-imdb/fused/train_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [ ]:
image_file_paths

In [ ]:
genre_labels

In [ ]:
unique_labels_set = set()

for labels in genre_labels:
    unique_labels_set.update(labels)

unique_labels = sorted(list(unique_labels_set))

In [ ]:
unique_labels

In [ ]:
len(unique_labels)

26

In [ ]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [ ]:
label_counts

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in unique_labels]
    multi_hot_labels.append(multi_hot)

In [ ]:
multi_hot_labels[1]

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [ ]:
genre_labels[1]

['Crime', 'Drama', 'Thriller']

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

# Define data transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create a custom dataset
train_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels, transform=transform)

In [ ]:
len(train_dataset)

46656

In [ ]:
train_dataset[0]

# Test Dataset Preparation

In [ ]:
import os
test_image_file_paths = []
test_genre_labels = []

image_folder_add = "/content/Dataset(s)/mm-imdb/fused/test"
labels_file = '/content/Dataset(s)/mm-imdb/fused/test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        test_image_file_paths.append(image_path)
        test_genre_labels.append(labels)

In [ ]:
# unique_labels_set = set()

# for labels in test_genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
# unique_labels

In [ ]:
len(unique_labels)

26

In [ ]:
test_multi_hot_labels = []

for labels in test_genre_labels:
    test_multi_hot = [1 if label in labels else 0 for label in unique_labels]
    test_multi_hot_labels.append(test_multi_hot)

In [ ]:
test_dataset = CustomMultiLabelDataset(test_image_file_paths, test_multi_hot_labels, transform=transform)

In [ ]:
len(test_dataset)

23397

# Training

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.resnet101(pretrained=True)  # You can use a pre-trained model

# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 512),
#     nn.ReLU(),
#     nn.Linear(512, len(unique_labels)),
#     nn.Sigmoid()
# )

model = models.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(unique_labels))
model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    if epoch > 15:
      save_dir = "/content/Model/Models-Train-21/"
      os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
      model_name = str(epoch) + "_model.pth"
      save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
      torch.save(model.state_dict(), save_path)

    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            # loss = criterion(outputs, labels.float())

            # Apply a threshold (e.g., 0.5) to convert logits to binary predictions
            predictions.extend((outputs > 0.5).int().cpu().numpy())
            true_labels.extend(labels.int().cpu().numpy())

    # Calculate evaluation metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(classification_report(true_labels, predictions))

Epoch 1/30, Loss: 0.24515408999063024


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi

Accuracy: 0.09082360986451254
Precision: 0.27835498477572496
Recall: 0.04454686297087706
F1-Score: 0.05403379672160902
              precision    recall  f1-score   support

           0       0.54      0.01      0.02      3132
           1       0.67      0.01      0.02      2463
           2       0.76      0.03      0.06       918
           3       0.00      0.00      0.00      1233
           4       0.66      0.15      0.24      7833
           5       0.62      0.01      0.01      3489
           6       0.00      0.00      0.00      1887
           7       0.63      0.75      0.68     12426
           8       0.74      0.03      0.05      1554
           9       0.00      0.00      0.00      1755
          10       0.00      0.00      0.00       306
          11       0.00      0.00      0.00      1035
          12       0.47      0.01      0.01      2475
          13       0.00      0.00      0.00       933
          14       0.00      0.00      0.00       759
          15    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi

Accuracy: 0.08253194854041117
Precision: 0.39690797472178324
Recall: 0.10114525517510352
F1-Score: 0.14737595398075803
              precision    recall  f1-score   support

           0       0.63      0.11      0.18      3132
           1       0.65      0.14      0.24      2463
           2       0.71      0.12      0.21       918
           3       0.00      0.00      0.00      1233
           4       0.80      0.14      0.23      7833
           5       0.71      0.21      0.33      3489
           6       0.73      0.53      0.61      1887
           7       0.85      0.31      0.45     12426
           8       0.87      0.06      0.11      1554
           9       0.64      0.01      0.02      1755
          10       0.00      0.00      0.00       306
          11       0.00      0.00      0.00      1035
          12       0.77      0.20      0.32      2475
          13       0.00      0.00      0.00       933
          14       0.00      0.00      0.00       759
          15    

...

Accuracy: 0.13600034192417831
Precision: 0.48276512073860456
Recall: 0.32669855392290625
F1-Score: 0.3803852293829708
              precision    recall  f1-score   support

           0       0.54      0.48      0.51      3132
           1       0.51      0.35      0.42      2463
           2       0.63      0.29      0.40       918
           3       0.34      0.10      0.15      1233
           4       0.68      0.49      0.57      7833
           5       0.63      0.45      0.52      3489
           6       0.79      0.51      0.62      1887
           7       0.69      0.76      0.73     12426
           8       0.45      0.47      0.46      1554
           9       0.48      0.30      0.37      1755
          10       0.37      0.11      0.17       306
          11       0.44      0.14      0.21      1035
          12       0.73      0.52      0.61      2475
          13       0.55      0.34      0.42       933
          14       0.32      0.15      0.20       759
          15     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi

Accuracy: 0.1243749198615207
Precision: 0.5085178679592662
Recall: 0.3205503711553183
F1-Score: 0.3685529498980363
              precision    recall  f1-score   support

           0       0.66      0.28      0.40      3132
           1       0.56      0.30      0.39      2463
           2       0.65      0.23      0.34       918
           3       0.33      0.20      0.25      1233
           4       0.71      0.41      0.52      7833
           5       0.60      0.48      0.53      3489
           6       0.75      0.53      0.62      1887
           7       0.69      0.74      0.71     12426
           8       0.72      0.15      0.25      1554
           9       0.49      0.22      0.30      1755
          10       0.23      0.25      0.24       306
          11       0.30      0.35      0.32      1035
          12       0.69      0.56      0.62      2475
          13       0.57      0.25      0.34       933
          14       0.33      0.09      0.14       759
          15       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi

Accuracy: 0.11916057614224046
Precision: 0.47479007002306484
Recall: 0.3322487373513934
F1-Score: 0.36664462016004296
              precision    recall  f1-score   support

           0       0.62      0.36      0.45      3132
           1       0.43      0.48      0.46      2463
           2       0.41      0.52      0.46       918
           3       0.41      0.07      0.12      1233
           4       0.60      0.62      0.61      7833
           5       0.55      0.57      0.56      3489
           6       0.80      0.52      0.63      1887
           7       0.73      0.61      0.66     12426
           8       0.33      0.63      0.44      1554
           9       0.44      0.33      0.37      1755
          10       0.33      0.08      0.12       306
          11       0.38      0.18      0.24      1035
          12       0.76      0.46      0.57      2475
          13       0.50      0.36      0.42       933
          14       0.33      0.10      0.16       759
          15     

# New Section

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

save_dir = "/content/Model/Models-Train-1"
load_path = os.path.join(save_dir, '8_model.pth')


model = models.resnet101(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, len(unique_labels)),
    nn.Sigmoid()
)
model.load_state_dict(torch.load(load_path))
model.eval()
model.to(device)

In [ ]:
# Initialize lists to store predicted and ground truth labels
predicted_labels = []
true_labels = []

# Iterate through the test set and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted_labels.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
predicted_labels[0]

In [ ]:
threshold = 0.5

binary_predictions = []

for sample_logits in predicted_labels:
    binary_sample_predictions = [1 if value >= threshold else 0 for value in sample_logits]
    binary_predictions.append(binary_sample_predictions)

In [ ]:
binary_predictions[1000]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(true_labels, binary_predictions)
precision = precision_score(true_labels, binary_predictions, average='macro')
recall = recall_score(true_labels, binary_predictions, average='macro')
f1 = f1_score(true_labels, binary_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(classification_report(true_labels, binary_predictions))